# Extraction referentiel France Travail

URL : 

https://francetravail.io/data/api/offres-emploi

https://francetravail.io/data/api/offres-emploi/documentation#/

## Imports

In [8]:
import http.client
import requests
import json
import pandas as pd
import os

In [9]:
# ---------------------------
# CONFIGURATION
# ---------------------------
# France Travail (ex-Pôle emploi)
# FT_CLIENT_ID = Defined in .env
# FT_CLIENT_SECRET = Defined in .env
# FT_SCOPE = Defined in .env

##################  VARIABLES  ##################
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

# Paramètres de recherche
JOB_QUERY = "data analyst"
COMMUNE = "78300"
DISTANCE = 500

# Nombre d'annonces par page requise
BLOC_PAGINATION = 50

# Nombre de pages max
MAX_PAGES = 1   # Limiter le nombre de pages récupérées

# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_referentiel(ref, token):               
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/referentiel/{ref}"
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    return pd.DataFrame(data)

# ---------------------------
# Extraction totale referentiel
# ---------------------------
def fetch_france_travail_referentiel_global(token):
    appellations = fetch_france_travail_referentiel("appellations", token)
    nafs = fetch_france_travail_referentiel("nafs", token)
    communes = fetch_france_travail_referentiel("communes", token)
    continents = fetch_france_travail_referentiel("departements", token)
    departements = fetch_france_travail_referentiel("departements", token)
    domaines = fetch_france_travail_referentiel("domaines", token)
    langues = fetch_france_travail_referentiel("langues", token)
    metiers = fetch_france_travail_referentiel("metiers", token)
    naturesContrats = fetch_france_travail_referentiel("naturesContrats", token)
    niveauxFormations = fetch_france_travail_referentiel("niveauxFormations", token)
    pays = fetch_france_travail_referentiel("pays", token)
    permis = fetch_france_travail_referentiel("permis", token)
    regions = fetch_france_travail_referentiel("regions", token)
    secteursActivites = fetch_france_travail_referentiel("secteursActivites", token)
    themes = fetch_france_travail_referentiel("themes", token)
    typesContrats = fetch_france_travail_referentiel("typesContrats", token)
    
    # Save into different sheets of the same Excel file
    with pd.ExcelWriter("../data/raw_data/my_data.xlsx") as writer:
        appellations.to_excel(writer, sheet_name="appellations", index=False)
        nafs.to_excel(writer, sheet_name="nafs", index=False)
        communes.to_excel(writer, sheet_name="communes", index=False)
        continents.to_excel(writer, sheet_name="continents", index=False)
        departements.to_excel(writer, sheet_name="departements", index=False)
        domaines.to_excel(writer, sheet_name="domaines", index=False)
        langues.to_excel(writer, sheet_name="langues", index=False)
        metiers.to_excel(writer, sheet_name="metiers", index=False)
        naturesContrats.to_excel(writer, sheet_name="naturesContrats", index=False)
        niveauxFormations.to_excel(writer, sheet_name="niveauxFormations", index=False)
        pays.to_excel(writer, sheet_name="pays", index=False)
        permis.to_excel(writer, sheet_name="permis", index=False)
        regions.to_excel(writer, sheet_name="regions", index=False)
        secteursActivites.to_excel(writer, sheet_name="secteursActivites", index=False)
        themes.to_excel(writer, sheet_name="themes", index=False)
        typesContrats.to_excel(writer, sheet_name="typesContrats", index=False)
    print("Récupération du référentiel France Travail SUCCES !!!")
    
# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération du référentiel France Travail...")
    fetch_france_travail_referentiel_global(token)

Authentification France Travail...
Récupération du référentiel France Travail...
Récupération du référentiel France Travail SUCCES !!!


In [10]:
        # appellations
        # nafs
        # communes
        # continents
        # departements
        # domaines
        # langues
        # metiers
        # naturesContrats
        # niveauxFormations
        # pays
        # permis
        # regions
        # secteursActivites
        # themes
        # typesContrats

In [11]:
# # ---------------------------
# # CONFIGURATION
# # ---------------------------
# # France Travail (ex-Pôle emploi)
# # FT_CLIENT_ID = Defined in .env
# # FT_CLIENT_SECRET = Defined in .env
# # FT_SCOPE = Defined in .env

# ##################  VARIABLES  ##################
# FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
# FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
# FT_SCOPE = os.environ.get("FT_SCOPE")

# # Paramètres de recherche
# JOB_QUERY = "data analyst"
# COMMUNE = "78300"
# DISTANCE = 500

# # Nombre d'annonces par page requise
# BLOC_PAGINATION = 50

# # Nombre de pages max
# MAX_PAGES = 1   # Limiter le nombre de pages récupérées

# # ---------------------------
# # AUTH FRANCE TRAVAIL
# # ---------------------------
# def get_ft_token():
#     url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
#     data = {
#         "grant_type": "client_credentials",
#         "client_id": FT_CLIENT_ID,
#         "client_secret": FT_CLIENT_SECRET,
#         "scope": FT_SCOPE,
#     }
#     r = requests.post(url, data=data)
#     r.raise_for_status()
#     return r.json()["access_token"]

# # ---------------------------
# # API CALL FRANCE TRAVAIL
# # ---------------------------
# def fetch_france_travail_referentiel(token):
#     referentiel = ["appellations", "nafs","communes","continents",
#                    "departements","domaines","langues","metiers",
#                    "naturesContrats","niveauxFormations","pays","permis",
#                     "regions","secteursActivites","themes","typesContrats"]                     
#     headers = {"Authorization": f"Bearer {token}"}
    
#     for ref in referentiel:
#         url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/referentiel/{ref}"
#         r = requests.get(url, headers=headers)
#         r.raise_for_status()
#         data = r.json()
#         ref_df=pd.DataFrame(data)

#         # Save into different sheets of the same Excel file
#         with pd.ExcelWriter("../data/raw_data/my_data.xlsx") as writer:
#             ref_df.to_excel(writer, sheet_name=f"{ref}", index=False)


#         # # Save into different sheets of the same Excel file
#         # with pd.ExcelWriter("../data/raw_data/my_data.xlsx", engine="openpyxl") as writer:
#         #     ref_df.to_excel(writer, sheet_name=f"{ref}", index=False)
#     return ''

# # ---------------------------
# # MAIN
# # ---------------------------
# if __name__ == "__main__":
#     print("Authentification France Travail...")
#     token = get_ft_token()

#     print("Récupération du référentiel France Travail...")
#     ft_referentiel = fetch_france_travail_referentiel(token)



In [12]:
# # ---------------------------
# # CONFIGURATION
# # ---------------------------
# # France Travail (ex-Pôle emploi)
# # FT_CLIENT_ID = Defined in .env
# # FT_CLIENT_SECRET = Defined in .env
# # FT_SCOPE = Defined in .env

# ##################  VARIABLES  ##################
# FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
# FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
# FT_SCOPE = os.environ.get("FT_SCOPE")

# # Paramètres de recherche
# JOB_QUERY = "data analyst"
# COMMUNE = "78300"
# DISTANCE = 500

# # Nombre d'annonces par page requise
# BLOC_PAGINATION = 50

# # Nombre de pages max
# MAX_PAGES = 1   # Limiter le nombre de pages récupérées

# # ---------------------------
# # AUTH FRANCE TRAVAIL
# # ---------------------------
# def get_ft_token():
#     url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
#     data = {
#         "grant_type": "client_credentials",
#         "client_id": FT_CLIENT_ID,
#         "client_secret": FT_CLIENT_SECRET,
#         "scope": FT_SCOPE,
#     }
#     r = requests.post(url, data=data)
#     r.raise_for_status()
#     return r.json()["access_token"]

# # ---------------------------
# # API CALL FRANCE TRAVAIL
# # ---------------------------
# def fetch_france_travail_referentiel(token, max_pages=MAX_PAGES):
#         # referentiel = ["appellations",
#         #               "nafs","communes","continents", 
#         #                 "departements", "domaines",
#         #                 "langues","metiers",
#         #                 "naturesConterats",
#         #                 "niveauxFormations","pays","permis",
#         #                 "regions","secteursActivites",
#         #                 "themes","typesContrats"
#         #               ]  

#     referentiel = ["appellations"]                     ] 
#     headers = {"Authorization": f"Bearer {token}"}
    
#     for ref in referentiel:
#             url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/referentiel/{ref}"
#             r = requests.get(url, headers=headers)
#             r.raise_for_status()
#             data = r.json()
#             offres = data.get("resultats", [])

            
#     return all_jobs

# # ---------------------------
# # MAIN
# # ---------------------------
# if __name__ == "__main__":
#     print("Authentification France Travail...")
#     token = get_ft_token()

#     print("Récupération des offres France Travail...")
#     ft_jobs = fetch_france_travail_referentiel(token)

#     # print("Affichage Extract offres France Travail...")
#     # df = pd.DataFrame(ft_jobs)
#     # display(df.head(20))

#     # # Export vers CSV
#     # df.to_csv("../data/raw_data/offres_emploi.csv", index=False, encoding="utf-8")
#     # print(f"{len(ft_jobs)} offres uniques exportées dans offres_emploi.csv ✅")
